# Delete SageMaker Resources

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import base64
import os
from time import sleep
from typing import Dict, List

import boto3
from dotenv import find_dotenv, load_dotenv

In [3]:
%aimport src.s3.buckets
import src.s3.buckets as s3b

%aimport src.iam.iam_roles
import src.iam.iam_roles as iamr

%aimport src.ec2.ec2_instances_sec_groups
import src.ec2.ec2_instances_sec_groups as ec2h

In [4]:
load_dotenv(find_dotenv())

True

In [5]:
aws_region = os.getenv("AWS_REGION")

## About

In this notebook, the following resources related to AWS SageMaker will be destroyed
- SageMaker notebook instance
- SageMaker notebook lifecycle configuration
- CloudWatch Logging group for the SageMaker instance
- EC2 Security group for the SageMaker instance
- S3 bucket

### Pre-Requisites
1. As mentioned in `README.md`, the following environment variables should be set with the user's AWS credendials ([1](https://docs.aws.amazon.com/sdk-for-php/v3/developer-guide/guide_credentials_environment.html), [2](https://docs.aws.amazon.com/sdk-for-php/v3/developer-guide/guide_credentials_profiles.html))
   - `AWS_ACCESS_KEY_ID`
   - `AWS_SECRET_KEY`
   - `AWS_REGION`

   These credentials must be associated to a user group whose users have been granted programmatic access to AWS resources. In order to configure this for an IAM user group, see the documentation [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html#id_users_create_console).

### Notes
1. All resources must exist in the same AWS region (specified by the environment variable `AWS_REGION`).

## User Inputs

In [6]:
# S3
s3_bucket_name = ""

# IAM
iam_role_name = "AmazonSageMaker-ExecutionRole-20211228T122046"
iam_policy_name = "AmazonSageMaker-ExecutionPolicy-20211228T122046"
delete_sagemaker_s3_iam_role = "no"

# EC2 Security Groups
sg_group_name = "mysgname"

# Sagemaker Lifecycle
nb_lifecycle_name = "mynbconfig"
nb_instance_name = "mydemo"
nb_instance_tags = [{"Key": "Name", "Value": nb_instance_name}]

# Cloud Watch
cw_log_group_name = "/aws/sagemaker/NotebookInstances"

## SageMaker

### Stop Notebook Instance

In [7]:
def stop_sagemaker_nb_instance(nb_instance_name: str, aws_region: str) -> None:
    client = boto3.client("sagemaker", region_name=aws_region)
    stop_nb_response = client.stop_notebook_instance(
        NotebookInstanceName=nb_instance_name
    )
    return stop_nb_response

In [ ]:
%%time
stop_nb_response = stop_sagemaker_nb_instance(nb_instance_name, aws_region)
stop_nb_response

In [10]:
sleep(90)

### Delete Notebook Instance

In [12]:
def delete_sagemaker_nb_instance(nb_instance_name: str, aws_region: str):
    client = boto3.client("sagemaker", region_name=aws_region)
    response_nb_list = client.list_notebook_instances(NameContains=nb_instance_name)
    nb_dict = response_nb_list["NotebookInstances"][0]
    nb_state = nb_dict["NotebookInstanceStatus"]
    if nb_state not in ["Stopped"]:
        print(
            f"Sagemaker Instance in state {nb_state}. Could not be deleted. Did nothing."
        )
    else:
        nb_delete_response = client.delete_notebook_instance(
            NotebookInstanceName=nb_instance_name
        )
        print("Sagemaker Instance Deleted.")

In [15]:
%%time
delete_sagemaker_nb_instance(nb_instance_name, aws_region)

Sagemaker Instance Deleted.
CPU times: user 20.6 ms, sys: 3.63 ms, total: 24.3 ms
Wall time: 236 ms


### Delete the CloudWatch Logging Group for SageMaker Instance

In [16]:
def delete_cw_log_group_stream(cw_log_group_name: str, aws_region: str) -> Dict:
    """Delete CloudWatch Logging Group."""
    cw_logs_client = boto3.client("logs", region_name=aws_region)
    cw_log_deletion_response = cw_logs_client.delete_log_group(
        logGroupName=cw_log_group_name
    )

In [17]:
%%time
delete_cw_log_group_stream(cw_log_group_name, aws_region)

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 170 ms


### Delete EC2 Security Group for the SageMaker Instance

In [18]:
def delete_sagemaker_nb_instance_security_group(
    nb_instance_name: str, aws_region: str, sg_group_id: str
) -> None:
    client = boto3.client("sagemaker", region_name=aws_region)
    response_nb_list = client.list_notebook_instances(NameContains=nb_instance_name)
    # if not response_nb_list["NotebookInstanceLifecycleConfigs"]:
    if not response_nb_list:
        print(
            "Found Notebook instance that depends on "
            f"Security Group: {sg_group_id}. Did nothing."
        )
    else:
        ec2h.delete_sg(sg_group_id, aws_region)

In [ ]:
%%time
sg_filter = dict(Filters=[{"Name": "tag:Name", "Values": [sg_group_name]}])
sg_id_list = ec2h.get_security_group_ids(aws_region, sg_filter)
delete_sagemaker_nb_instance_security_group(nb_instance_name, aws_region, sg_id_list[0])

## IAM

### (Optional) Delete IAM Sagemaker-S3 Policy

Get Attached IAM Policies

In [ ]:
if delete_sagemaker_s3_iam_role == "yes":
    iam_sagemaker_s3_policy_list = iamr.get_iam_policies(aws_region, attached=True)

Filter list of policies to get user-defined S3 access IAM policy used by SageMaker IAM role

In [ ]:
%%time
if delete_sagemaker_s3_iam_role == "yes":
    iam_policy_list = [
        iam_sagemaker_s3_policy
        for iam_sagemaker_s3_policy in iam_sagemaker_s3_policy_list
        if iam_sagemaker_s3_policy["PolicyName"] == iam_policy_name
    ]
    iam_policy_dict = iam_policy_list[0]
    print(iam_policy_list)
    print(iam_policy_dict)

Detach IAM Sagemaker-S3 Policy from the SageMaker IAM role and Delete the policy

In [ ]:
%%time
if delete_sagemaker_s3_iam_role == "yes":
    policy_detachment_response, policy_deletion_response = iamr.delete_iam_policy(
        iam_policy_dict["Arn"], iam_role_name, aws_region
    )
    print(policy_detachment_response)
    print(policy_deletion_response)

Detach the AWS-provided policy `AmazonSageMakerFullAccess` from the SageMaker IAM role

In [ ]:
%%time
if delete_sagemaker_s3_iam_role == "yes":
    iam_policy_list = [
        iam_sagemaker_s3_policy
        for iam_sagemaker_s3_policy in iam_sagemaker_s3_policy_list
        if iam_sagemaker_s3_policy["PolicyName"] == "AmazonSageMakerFullAccess"
    ]
    iam_policy_dict = iam_policy_list[0]
    policy_detachment_response, policy_deletion_response = iamr.delete_iam_policy(
        iam_policy_dict["Arn"], iam_role_name, aws_region, delete=False
    )
    print(policy_detachment_response)
    print(policy_deletion_response)

In [37]:
sleep(15)

### (Optional) Delete IAM Role

In [38]:
%%time
iam_role_deletion_response = iamr.delete_iam_role(iam_role_name, aws_region)
iamr.check_iam_role_deletion(iam_role_name, aws_region)

CPU times: user 67.1 ms, sys: 4 µs, total: 67.1 ms
Wall time: 424 ms


## (Optional) Delete the S3 Bucket

If a name was provided for the S3 bucket, then delete it

In [39]:
%%time
if s3_bucket_name:
    s3b.delete_s3_bucket(s3_bucket_name, aws_region)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs
